# Does Specializing More than Once Improve Learning

In [1]:
from res_specialization import *
from matplotlib import pyplot as plt
import sys
import random
import pickle
plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
TRIALS      = 1000
NUM_TO_SPEC = 3
TOL = 5

diff_eq_params = {"x0": [-20, 10, -.5], 
                  "begin": 0, 
                  "end": 60, 
                  "timesteps":60000}

res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

results = dict()


def save_results():
    pickle.dump(results, open("spec_tw.pkl","wb"))
# end

### Model trajectory

In [3]:
t, train_t, u = lorenz_equ(**diff_eq_params )
train_t = t[30000:55000]
test_t  = t[55000:]

### Re-run grant figure trials to get prediction length


In [4]:
random_err       = []
random_pred      = []
specialized_err  = []
specialized_pred = []


spec_sizes       = []
spec_edges       = []

for i in range(TRIALS):
    
    ######################
    # Random graph
    ######################
    
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Make predicitions
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store Error
    random_err.append(L2error(u(train_t), train_pre))
    random_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    
    #######################
    # Specialize graph
    ######################
    
    A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
    spec_sizes.append(A.shape[0])
    spec_edges.append(np.sum(A != 0))
    
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    specialized_err.append(L2error(u(train_t), train_pre))
    specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [5]:
results["random_err"]       = random_err
results["random_pred"]      = random_pred
results["specialized_err"]  = specialized_err
results["specialized_pred"] = specialized_pred
results["spec_sizes"]       = spec_sizes
results["spec_edges"]       = spec_edges
save_results()
print("Mean Prediction Length: {}".format(np.mean(random_pred)))
print("Mean Prediction Length: {}".format(np.mean(specialized_pred)))

Mean Prediction Length: 388.438
Mean Prediction Length: 883.659


### Control: Same sizes and same edge densities as previous experiment

In [6]:
match_edge_err = []
match_edge_pred = []

for s,e in zip(spec_sizes,spec_edges):
    # Test random graph with same number of edges as specialized graphs
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    match_edge_err.append(L2error(u(train_t), train_pre))
    match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [7]:
results["match_edge_err"] = match_edge_err
results["match_edge_pred"] = match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(match_edge_pred)))
save_results()


Mean Prediction Length: 733.477


### Specialization without finding best nodes

In [8]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

random_specialized_err  = []
random_specialized_pred = []
rspec_sizes         = []
rspec_edges         = []

for i in range(TRIALS):
    # Random graph
    rc = ResComp(3,3,**res_params)
    
    # Turn reservoir to integer adj matrix
    A = rc.res
    for j in range(A.shape[0]): A[j,j] = 0
    A = (A != 0)*1
    
    # Specialize the reservoir
    A = specializeGraph(A, random.sample(list(range(30)),27))
    
    # Store data
    rspec_sizes.append(A.shape[0])
    rspec_edges.append(np.sum(A != 0))
    
    # Make new reservoir
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Get predicitons
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store data
    random_specialized_err.append(L2error(u(train_t), train_pre))
    random_specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [9]:
results["random_specialized_err"]  = random_specialized_err
results["random_specialized_pred"] = random_specialized_pred
results["rspec_sizes"]             = rspec_sizes
results["rspec_edges"]             = rspec_edges

print("Mean Prediction Length: {}".format(np.mean(random_specialized_pred)))
save_results()


Mean Prediction Length: 813.484


### Control: Same sizes and same edge densities as previous experiment


In [10]:
rand_match_edge_err  = []
rand_match_edge_pred = []

for s,e in zip(rspec_sizes, rspec_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    rand_match_edge_err.append(L2error(u(train_t), train_pre))
    rand_match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    

In [11]:
results["rand_match_edge_err"]  = rand_match_edge_err
results["rand_match_edge_pred"] = rand_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(rand_match_edge_pred)))
save_results()


Mean Prediction Length: 685.279


### Targeted Specialize Twice

In [12]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

spec_tw_pre = []
spec_tw_err = []
tw_edges = []
tw_sizes = []
HOW_MANY_SPEC = 2

for i in range(TRIALS):
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 2 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        rc.fit(train_t,u)
    
    # Predict the system states
    tw_sizes.append(A.shape[0])
    tw_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_tw_err.append(L2error(u(train_t), train_pre))
    spec_tw_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))

In [13]:
results["spec_tw_pre"] = spec_tw_pre
results["spec_tw_err"] = spec_tw_err
results["tw_edges"] = tw_edges
results["tw_sizes"] = tw_sizes
save_results()


In [14]:
print("Mean Prediction Length: {}".format(np.mean(spec_tw_pre)))


Mean Prediction Length: 932.171


### Control: Same sizes and same edge densities as previous experiment

In [15]:
tw_match_edge_err  = []
tw_match_edge_pred = []

for s,e in zip(tw_sizes, tw_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    tw_match_edge_err.append(L2error(u(train_t), train_pre))
    tw_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [16]:
results["tw_match_edge_err"] = tw_match_edge_err
results["tw_match_edge_pred"] = tw_match_edge_pred
save_results()
print("Mean Prediction Length: {}".format(np.mean(tw_match_edge_pred)))


Mean Prediction Length: 782.524


### Targeted Specialize Three Times

In [17]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

spec_thr_pre = []
spec_thr_err = []
thr_edges = []
thr_sizes = []
HOW_MANY_SPEC = 3

for i in range(TRIALS):
    # Make RC
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 3 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        rc.fit(train_t,u)
    
    # Make predicitons
    thr_sizes.append(A.shape[0])
    thr_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_thr_err.append(L2error(u(train_t), train_pre))
    spec_thr_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))


In [18]:
results["spec_thr_pre"] = spec_thr_pre
results["spec_thr_err"] = spec_thr_err
results["thr_edges"] = thr_edges
results["thr_sizes"] = thr_sizes
print("Mean Prediction Length: {}".format(np.mean(spec_thr_pre)))

save_results()


Mean Prediction Length: 1000.392


### Control: Same sizes and same edge densities as previous experiment

In [19]:
thr_match_edge_err  = []
thr_match_edge_pred = []

for s,e in zip(thr_sizes, thr_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict systems
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    thr_match_edge_err.append(L2error(u(train_t), train_pre))
    thr_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [20]:
results["thr_match_edge_err"] = thr_match_edge_err
results["thr_match_edge_pred"] = thr_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(thr_match_edge_pred)))

save_results()


Mean Prediction Length: 791.472


## Organize results into dataframes


In [21]:
import pandas as pd

In [22]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
df = pd.DataFrame(results)
df.describe()

,random_err,random_pred,specialized_err,specialized_pred,spec_sizes,spec_edges,match_edge_err,match_edge_pred,random_specialized_err,random_specialized_pred,...,tw_edges,tw_sizes,tw_match_edge_err,tw_match_edge_pred,spec_thr_pre,spec_thr_err,thr_edges,thr_sizes,thr_match_edge_err,thr_match_edge_pred
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,15871.242874,388.438000,25.238542,883.659000,114.412000,233.937000,271.277332,733.477000,38.115356,813.484000,...,287.179000,146.333000,114.428762,782.524000,1000.392000,14.873572,354.458000,185.562000,80.054563,791.472000
std,22477.284112,270.971108,115.470592,300.337506,68.978811,114.052062,985.248315,193.064003,164.503863,215.001976,...,163.213699,95.006085,364.835290,233.000379,550.558341,15.120622,227.409915,130.870812,344.585998,189.926942
min,14.294774,0.000000,7.791033,478.000000,33.000000,94.000000,4.972284,0.000000,7.366969,381.000000,...,103.000000,42.000000,3.723034,2.000000,539.000000,7.205787,120.000000,49.000000,3.318782,0.000000
25%,97.632089,121.000000,13.818589,781.000000,71.000000,165.000000,14.134378,646.000000,15.059030,716.000000,...,191.000000,88.000000,12.604063,690.000000,844.750000,12.419141,217.750000,108.000000,11.547124,698.000000
50%,6995.487241,424.000000,15.388949,860.000000,94.000000,205.000000,17.187648,737.500000,17.128110,800.000000,...,242.000000,119.000000,15.567151,782.500000,917.000000,14.094632,288.000000,150.500000,14.319591,784.000000
75%,23239.336804,601.000000,17.824807,940.000000,133.000000,265.000000,43.357179,830.250000,18.999374,889.250000,...,331.250000,174.000000,19.925865,866.500000,985.250000,15.897186,405.250000,217.000000,17.932136,869.000000
max,163560.581180,1526.000000,2028.605416,5000.000000,642.000000,1160.000000,14387.515894,3108.000000,2751.088949,4169.000000,...,2279.000000,1294.000000,3917.110643,5000.000000,5000.000000,479.931708,2179.000000,1381.000000,5487.324863,4241.000000


### Error

In [23]:
df[["random_specialized_err","specialized_err","spec_tw_err", "spec_thr_err"]].describe()

,random_specialized_err,specialized_err,spec_tw_err,spec_thr_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,38.115356,25.238542,18.351314,14.873572
std,164.503863,115.470592,77.423470,15.120622
min,7.366969,7.791033,6.044192,7.205787
25%,15.059030,13.818589,12.914267,12.419141
50%,17.128110,15.388949,14.659355,14.094632
75%,18.999374,17.824807,16.718585,15.897186
max,2751.088949,2028.605416,2359.864260,479.931708


### Error Controls

In [24]:
df[["rand_match_edge_err", "match_edge_err", "tw_match_edge_err", "thr_match_edge_err"]].describe()

,rand_match_edge_err,match_edge_err,tw_match_edge_err,thr_match_edge_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,908.703480,271.277332,114.428762,80.054563
std,2932.712067,985.248315,364.835290,344.585998
min,5.349209,4.972284,3.723034,3.318782
25%,16.135271,14.134378,12.604063,11.547124
50%,19.648337,17.187648,15.567151,14.319591
75%,394.475265,43.357179,19.925865,17.932136
max,39548.147100,14387.515894,3917.110643,5487.324863


### Prediction

In [25]:
df[["random_specialized_pred","specialized_pred","spec_tw_pre", "spec_thr_pre"]].describe()

,random_specialized_pred,specialized_pred,spec_tw_pre,spec_thr_pre
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,813.484000,883.659000,932.17100,1000.392000
std,215.001976,300.337506,422.15906,550.558341
min,381.000000,478.000000,534.00000,539.000000
25%,716.000000,781.000000,818.75000,844.750000
50%,800.000000,860.000000,884.00000,917.000000
75%,889.250000,940.000000,960.00000,985.250000
max,4169.000000,5000.000000,5000.00000,5000.000000


### Prediction Control

In [26]:
df[["rand_match_edge_pred", "match_edge_pred", "tw_match_edge_pred", "thr_match_edge_pred"]].describe()

,rand_match_edge_pred,match_edge_pred,tw_match_edge_pred,thr_match_edge_pred
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,685.279000,733.477000,782.524000,791.472000
std,260.464602,193.064003,233.000379,189.926942
min,0.000000,0.000000,2.000000,0.000000
25%,603.500000,646.000000,690.000000,698.000000
50%,702.000000,737.500000,782.500000,784.000000
75%,793.250000,830.250000,866.500000,869.000000
max,5000.000000,3108.000000,5000.000000,4241.000000


### Sizes

In [27]:
sizes = df[["rspec_sizes", "spec_sizes", "tw_sizes", "thr_sizes"]]
sizes.describe()

,rspec_sizes,spec_sizes,tw_sizes,thr_sizes
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,76.802000,114.412000,146.333000,185.562000
std,39.819648,68.978811,95.006085,130.870812
min,27.000000,33.000000,42.000000,49.000000
25%,52.000000,71.000000,88.000000,108.000000
50%,66.000000,94.000000,119.000000,150.500000
75%,86.250000,133.000000,174.000000,217.000000
max,390.000000,642.000000,1294.000000,1381.000000


### Edges

In [28]:
edges = df[["rspec_edges", "spec_edges", "tw_edges", "thr_edges"]]
edges.describe()

,rspec_edges,spec_edges,tw_edges,thr_edges
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,175.547000,233.937000,287.179000,354.458000
std,69.765687,114.052062,163.213699,227.409915
min,79.000000,94.000000,103.000000,120.000000
25%,133.750000,165.000000,191.000000,217.750000
50%,161.000000,205.000000,242.000000,288.000000
75%,195.000000,265.000000,331.250000,405.250000
max,803.000000,1160.000000,2279.000000,2179.000000


### Connectivity: (num edges)/(num nodes)^2

In [29]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
edge_prob = pd.DataFrame(np.array(edges)/(np.array(sizes)**2),columns=labels)
edge_prob.describe()

,random_specialized,specialized,spec_twice,spec_three
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.038007,0.024463,0.018413,0.014616
std,0.017324,0.012484,0.009563,0.008144
min,0.005279,0.002814,0.001361,0.001103
25%,0.025776,0.014669,0.011069,0.008592
50%,0.036757,0.023128,0.017072,0.012929
75%,0.048515,0.032556,0.024166,0.018938
max,0.122085,0.086318,0.065599,0.053728


In [30]:
df[["random_err", "random_pred"]].describe()

,random_err,random_pred
count,1000.000000,1000.000000
mean,15871.242874,388.438000
std,22477.284112,270.971108
min,14.294774,0.000000
25%,97.632089,121.000000
50%,6995.487241,424.000000
75%,23239.336804,601.000000
max,163560.581180,1526.000000
